<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

# Machine Learning Avanzado
*2 de Diciembre de 2024*

#### Integrantes: 
` Vicente Gallardo, Giuseppe Lavarello, `

## 1. Objetivo

El objetivo de este laboratorio es ajustar redes neuronales convolucionales según lo aprendido en clase para desarrollar un análisis de datos.

Para esto, se trabajará con la base de **Chihuahuas vs Muffins** disponible en **Kaggle** ([enlace al dataset](https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification)) y se realizará lo siguiente:

1. Implementar un modelo de Perceptrón Multicapa (MLP) para la clasificación de las imágenes.  
2. Implementar y ajustar dos arquitecturas diferentes de Redes Neuronales Convolucionales (CNN).  
3. Comparar los resultados entre el MLP y las CNN en términos de precisión y desempeño general.  



## 2. Introducción

En el ámbito de la ciencia de datos y el aprendizaje profundo, la clasificación de imágenes es un problema comun con amplias aplicaciones prácticas, como el reconocimiento facial, la detección de objetos y el diagnóstico médico. Este laboratorio aborda el desafío de clasificar imágenes en dos categorías visualmente similares: **chihuahuas** y **muffins**. 

Para esto se utilizará el conjunto de datos **Chihuahuas vs Muffins** de Kaggle, que proporciona imágenes etiquetadas para entrenar y evaluar modelos de clasificación.

En este informe, se comparará el desempeño de un modelo de Perceptrón Multicapa (MLP) con dos arquitecturas diferentes de redes convolucionales, evaluando métricas clave como precisión y robustez del modelo. Además, se explorará cómo cada enfoque aborda el desafío y se destacarán las fortalezas y limitaciones de las técnicas utilizadas.


## 3. Metodología 

1. Obtención del Dataset:  
    -   Kaggle (Chihuahuas vs Muffins). 

2. Preprocesamiento:  
    - Carga y exploración de imágenes.   
    - Normalización de datos.   
    - División en conjuntos de entrenamiento, validación y prueba.   

3. Modelos a implementar:   
    - Modelo 1: Perceptrón Multicapa (MLP).   
    - Modelo 2: Red Convolucional simple (CNN básica).   
    - Modelo 3: Red Convolucional más profunda con más capas convolucionales.   

4. Métricas de evaluación:   
    - Precisión (accuracy).   
    - Matriz de confusión.   
    - AUC-ROC.   

5. Interpretación de los resultados y conclusiones. 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import models, layers, optimizers, activations

2024-11-28 20:33:58.388974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 20:33:58.424646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 20:33:58.435497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 20:33:58.580599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
